<a href="https://colab.research.google.com/github/Arminsbss/Simplicial-Neural-Network/blob/main/impute_citations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional
import torch.utils.data as data
import numpy as np
import sys

!git clone https://github.com/stefaniaebli/simplicial_neural_networks.git


Cloning into 'simplicial_neural_networks'...
remote: Enumerating objects: 339, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 339 (delta 50), reused 48 (delta 48), pack-reused 280
Receiving objects: 100% (339/339), 2.42 MiB | 1.71 MiB/s, done.
Resolving deltas: 100% (189/189), done.


In [ ]:
# %cd /content/simplicial_neural_networks/scnn
# from simplicial_neural_networks import scnn
import simplicial_neural_networks.scnn
import simplicial_neural_networks.scnn.chebyshev


In [ ]:
#@title scnn
#scnn
import torch
import torch.nn as nn
import numpy as np
import scipy.sparse as sp

import simplicial_neural_networks.scnn.chebyshev
def coo2tensor(A):
    assert(sp.isspmatrix_coo(A))
    idxs = torch.LongTensor(np.vstack((A.row, A.col)))
    vals = torch.FloatTensor(A.data)
    return torch.sparse_coo_tensor(idxs, vals, size = A.shape, requires_grad = False)

class SimplicialConvolution(nn.Module):
    def __init__(self, K, C_in, C_out, enable_bias = True, variance = 1.0, groups = 1):
        assert groups == 1, "Only groups = 1 is currently supported."
        super().__init__()

        assert(C_in > 0)
        assert(C_out > 0)
        assert(K > 0)

        self.C_in = C_in
        self.C_out = C_out
        self.K = K
        self.enable_bias = enable_bias

        self.theta = nn.parameter.Parameter(variance*torch.randn((self.C_out, self.C_in, self.K)))
        if self.enable_bias:
            self.bias = nn.parameter.Parameter(torch.zeros((1, self.C_out, 1)))
        else:
            self.bias = 0.0

    def forward(self, L, x):
        assert(len(L.shape) == 2)
        assert(L.shape[0] == L.shape[1])

        (B, C_in, M) = x.shape

        assert(M == L.shape[0])
        assert(C_in == self.C_in)

        X = scnn.chebyshev.assemble(self.K, L, x)
        y = torch.einsum("bimk,oik->bom", (X, self.theta))
        assert(y.shape == (B, self.C_out, M))

        return y + self.bias

# This class does not yet implement the
# Laplacian-power-pre/post-composed with the coboundary. It can be
# simulated by just adding more layers anyway, so keeping it simple
# for now.
#
# Note: You can use this for a adjoints of coboundaries too. Just feed
# a transposed D.
class Coboundary(nn.Module):
    def __init__(self, C_in, C_out, enable_bias = True, variance = 1.0):
        super().__init__()

        assert(C_in > 0)
        assert(C_out > 0)

        self.C_in = C_in
        self.C_out = C_out
        self.enable_bias = enable_bias

        self.theta = nn.parameter.Parameter(variance*torch.randn((self.C_out, self.C_in)))
        if self.enable_bias:
            self.bias = nn.parameter.Parameter(torch.zeros((1, self.C_out, 1)))
        else:
            self.bias = 0.0

    def forward(self, D, x):
        assert(len(D.shape) == 2)

        (B, C_in, M) = x.shape

        assert(D.shape[1] == M)
        assert(C_in == self.C_in)

        N = D.shape[0]

        # This is essentially the equivalent of chebyshev.assemble for
        # the convolutional modules.
        X = []
        for b in range(0, B):
            X12 = []
            for c_in in range(0, self.C_in):
                X12.append(D.mm(x[b, c_in, :].unsqueeze(1)).transpose(0,1)) # D.mm(x[b, c_in, :]) has shape Nx1
            X12 = torch.cat(X12, 0)

            assert(X12.shape == (self.C_in, N))
            X.append(X12.unsqueeze(0))

        X = torch.cat(X, 0)
        assert(X.shape == (B, self.C_in, N))

        y = torch.einsum("oi,bin->bon", (self.theta, X))
        assert(y.shape == (B, self.C_out, N))

        return y + self.bias

In [ ]:
#@title chebyshef
#chebyshef
import torch
import scipy.sparse as sp
import scipy.sparse.linalg as spl
import numpy as np
def normalize(L, half_interval = False):
    assert(sp.isspmatrix(L))
    M = L.shape[0]
    assert(M == L.shape[1])
    topeig = spl.eigsh(L, k=1, which="LM", return_eigenvectors = False)[0]
    #print("Topeig = %f" %(topeig))

    ret = L.copy()
    if half_interval:
        ret *= 1.0/topeig
    else:
        ret *= 2.0/topeig
        ret.setdiag(ret.diagonal(0) - np.ones(M), 0)

    return ret

def assemble(K, L, x):
    (B, C_in, M) = x.shape
    assert(L.shape[0] == M)
    assert(L.shape[0] == L.shape[1])
    assert(K > 0)

    X = []
    for b in range(0, B):
        X123 = []
        for c_in in range(0, C_in):
            X23 = []
            X23.append(x[b, c_in, :].unsqueeze(1)) # Constant, k = 0 term.

            if K > 1:
                X23.append(L.mm(X23[0]))
            for k in range(2, K):
                X23.append(2*(L.mm(X23[k-1])) - X23[k-2])

            X23 = torch.cat(X23, 1)
            assert(X23.shape == (M, K))
            X123.append(X23.unsqueeze(0))

        X123 = torch.cat(X123, 0)
        assert(X123.shape == (C_in, M, K))
        X.append(X123.unsqueeze(0))

    X = torch.cat(X, 0)
    assert(X.shape == (B, C_in, M, K))

    return X

In [ ]:
#@title MySCNN

class MySCNN(nn.Module):
    def __init__(self, colors = 1):
        super().__init__()

        assert(colors > 0)
        self.colors = colors

        num_filters = 30 #20
        variance = 0.01 #0.001

        # Degree 0 convolutions.
        self.C0_1 = scnn.scnn.SimplicialConvolution(5, self.colors, num_filters*self.colors, variance=variance)
        self.C0_2 = scnn.scnn.SimplicialConvolution(5, num_filters*self.colors, num_filters*self.colors, variance=variance)
        self.C0_3 = scnn.scnn.SimplicialConvolution(5, num_filters*self.colors, self.colors, variance=variance)

        # Degree 1 convolutions.
        self.C1_1 = scnn.scnn.SimplicialConvolution(5, self.colors, num_filters*self.colors, variance=variance)
        self.C1_2 = scnn.scnn.SimplicialConvolution(5, num_filters*self.colors, num_filters*self.colors, variance=variance)
        self.C1_3 = scnn.scnn.SimplicialConvolution(5, num_filters*self.colors, self.colors, variance=variance)

        # Degree 2 convolutions.
        self.C2_1 = scnn.scnn.SimplicialConvolution(5, self.colors, num_filters*self.colors, variance=variance)
        self.C2_2 = scnn.scnn.SimplicialConvolution(5, num_filters*self.colors, num_filters*self.colors, variance=variance)
        self.C2_3 = scnn.scnn.SimplicialConvolution(5, num_filters*self.colors, self.colors, variance=variance)



    def forward(self, Ls, Ds, adDs, xs):
        assert(len(xs) == 3) # The three degrees are fed together as a list.

        assert(len(Ls) == len(Ds))
        Ms = [L.shape[0] for L in Ls]
        Ns = [D.shape[0] for D in Ds]

        Bs = [x.shape[0] for x in xs]
        C_ins = [x.shape[1] for x in xs]
        Ms = [x.shape[2] for x in xs]

        assert(Ms == [D.shape[1] for D in Ds])
        assert(Ms == [L.shape[1] for L in Ls])
        assert([adD.shape[0] for adD in adDs] == [D.shape[1] for D in Ds])
        assert([adD.shape[1] for adD in adDs] == [D.shape[0] for D in Ds])

        assert(Bs == len(Bs)*[Bs[0]])
        assert(C_ins == len(C_ins)*[C_ins[0]])

        out0_1 = self.C0_1(Ls[0], xs[0]) #+ self.D10_1(xs[1])
        out1_1 = self.C1_1(Ls[1], xs[1]) #+ self.D01_1(xs[0]) + self.D21_1(xs[2])
        out2_1 = self.C2_1(Ls[2], xs[2]) #+ self.D12_1(xs[1])

        out0_2 = self.C0_2(Ls[0], nn.LeakyReLU()(out0_1)) #+ self.D10_2(nn.LeakyReLU()(out1_1))
        out1_2 = self.C1_2(Ls[1], nn.LeakyReLU()(out1_1)) #+ self.D01_2(nn.LeakyReLU()(out0_1)) + self.D21_2(nn.LeakyReLU()(out2_1))
        out2_2 = self.C2_2(Ls[2], nn.LeakyReLU()(out2_1)) #+ self.D12_2(nn.LeakyReLU()(out1_1))

        out0_3 = self.C0_3(Ls[0], nn.LeakyReLU()(out0_2)) #+ self.D10_3(nn.LeakyReLU()(out1_2))
        out1_3 = self.C1_3(Ls[1], nn.LeakyReLU()(out1_2)) #+ self.D01_3(nn.LeakyReLU()(out0_2)) + self.D21_2(nn.LeakyReLU()(out2_2))
        out2_3 = self.C2_3(Ls[2], nn.LeakyReLU()(out2_2)) #+ self.D12_3(nn.LeakyReLU()(out1_2))

        #return [out0_3, torch.zeros_like(xs[1]), torch.zeros_like(xs[2])]
        #return [torch.zeros_like(xs[0]), out1_3, torch.zeros_like(xs[2])]
        return [out0_3, out1_3, out2_3]

In [ ]:
import sys
sys.path.append('.')

In [ ]:
%cd /content/simplicial_neural_networks/data

/content/simplicial_neural_networks/data


In [ ]:
logdir

NameError: ignored

In [ ]:
%cd /content/simplicial_neural_networks/data/s2_3_collaboration_complex
laplacians = np.load('150250_laplacians.npy',allow_pickle=True)

/content/simplicial_neural_networks/data/s2_3_collaboration_complex


In [ ]:
print(laplacians)

[<352x352 sparse matrix of type '<class 'numpy.float32'>'
 	with 3300 stored elements in COOrdinate format>
 <1474x1474 sparse matrix of type '<class 'numpy.float32'>'
 	with 25466 stored elements in COOrdinate format>
 <3285x3285 sparse matrix of type '<class 'numpy.float32'>'
 	with 4697 stored elements in COOrdinate format>
 <5019x5019 sparse matrix of type '<class 'numpy.float32'>'
 	with 5419 stored elements in COOrdinate format>
 <5559x5559 sparse matrix of type '<class 'numpy.float32'>'
 	with 5601 stored elements in COOrdinate format>
 <4547x4547 sparse matrix of type '<class 'numpy.float32'>'
 	with 4547 stored elements in COOrdinate format>
 <2732x2732 sparse matrix of type '<class 'numpy.float32'>'
 	with 2732 stored elements in COOrdinate format>
 <1175x1175 sparse matrix of type '<class 'numpy.float32'>'
 	with 1175 stored elements in COOrdinate format>
 <343x343 sparse matrix of type '<class 'numpy.float32'>'
 	with 343 stored elements in COOrdinate format>
 <61x61 sparse

In [ ]:
print(laplacians[1])

  (0, 0)	6.0
  (0, 63)	-1.0
  (0, 64)	-1.0
  (0, 65)	-1.0
  (0, 66)	-1.0
  (0, 67)	-1.0
  (0, 68)	-1.0
  (0, 69)	-1.0
  (0, 70)	-1.0
  (0, 71)	-1.0
  (0, 72)	-1.0
  (0, 73)	-1.0
  (0, 74)	-1.0
  (0, 75)	-1.0
  (0, 76)	-1.0
  (0, 77)	-1.0
  (0, 78)	-1.0
  (0, 79)	-1.0
  (0, 80)	-1.0
  (0, 81)	-1.0
  (0, 82)	-1.0
  (0, 83)	-1.0
  (0, 84)	-1.0
  (0, 85)	-1.0
  (0, 86)	-1.0
  :	:
  (1471, 1471)	11.0
  (1471, 139)	1.0
  (1471, 470)	1.0
  (1471, 555)	1.0
  (1471, 804)	1.0
  (1471, 1201)	1.0
  (1471, 1357)	1.0
  (1471, 1373)	1.0
  (1471, 1408)	1.0
  (1471, 1473)	-1.0
  (1472, 1472)	7.0
  (1473, 29)	-1.0
  (1473, 154)	-1.0
  (1473, 470)	-1.0
  (1473, 555)	-1.0
  (1473, 771)	-1.0
  (1473, 804)	-1.0
  (1473, 1087)	-1.0
  (1473, 1213)	-1.0
  (1473, 1294)	-1.0
  (1473, 1320)	-1.0
  (1473, 1373)	-1.0
  (1473, 1408)	-1.0
  (1473, 1471)	-1.0
  (1473, 1473)	7.0


In [ ]:
# the last one
print(laplacians[1].getrow(1471).getcol(1409).toarray()[0][0])
if int(laplacians[1].getrow(1471).getcol(1409).toarray()[0][0])==0.0:
  print(i)

0.0
1472


In [ ]:
laplacians[1].getrow(1471).getcol(1409).toarray()[0][0]==0

True

In [ ]:
print(type(laplacians[1]))

<class 'scipy.sparse._coo.coo_matrix'>


In [ ]:
laplacians[1].data[(laplacians[1].row==1471) & (laplacians[1].col==1409)]=9.0

In [ ]:
for i in range(1473):
  print(laplacians[1].getrow(i).getcol(i).toarray()[0][0])

6.0
6.0
6.0
6.0
6.0
9.0
9.0
9.0
9.0
9.0
9.0
9.0
9.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
11.0
11.0
11.0
11.0
11.0
11.0
11.0
11.0
11.0
11.0
9.0
9.0
9.0
9.0
9.0
9.0
9.0
9.0
6.0
6.0
6.0
6.0
6.0
9.0
9.0
9.0
9.0
9.0
9.0
9.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
9.0
9.0
9.0
9.0
9.0
9.0
5.0
5.0
5.0
6.0
9.0
9.0
8.0
9.0
7.0
4.0
9.0
9.0
9.0
7.0
5.0
2.0
3.0
5.0
7.0
9.0
4.0
7.0
4.0
8.0
7.0
9.0
8.0
7.0
7.0
7.0
8.0
7.0
7.0
2.0
9.0
8.0
8.0
6.0
9.0
6.0
8.0
9.0
6.0
9.0
7.0
8.0
9.0
6.0
7.0
6.0
6.0
8.0
7.0
6.0
8.0
6.0
7.0
9.0
3.0
7.0
9.0
8.0
7.0
9.0
5.0
7.0
7.0
6.0
7.0
8.0
6.0
5.0
7.0
8.0
6.0
6.0
7.0
7.0
8.0
5.0
7.0
6.0
7.0
11.0
11.0
11.0
11.0
11.0
11.0
11.0
11.0
11.0
5.0
5.0
5.0
9.0
9.0
9.0
9.0
9.0
9.0
9.0
9.0
4.0
6.0
9.0
6.0
6.0
7.0
6.0
6.0
7.0
6.0
4.0
7.0
6.0
6.0
9.0
9.0
6.0
9.0
9.0
9.0
7.0
7.0
9.0
9.0
6.0
9.0
9.0
6.0
6.0
6.0
7.0
9.0
9.0
6.0
9.0
9.0
4.0
9.0
5.0
5.0
5.0
5.0
5.0
9.0
9.0
9.0
9.0
9.0
9.0
9.0
9.0
8.0
8.0
8.0
8.0
8.0
8.0
11.0
11.0
11.0
11.0
11.0
11.0
11.0
11.0
11.0
11.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0

In [ ]:
import scipy.sparse as sp

# create new coordinates and values
new_coords = [(1471, 1409)]
new_values = [4.0]

# find the indices of the existing coordinates
idx = laplacians[1].row * laplacians[1].shape[1] + laplacians[1].col

# find the indices of the new coordinates
new_idx = [coord[0] * laplacians[1].shape[1] + coord[1] for coord in new_coords]

# find the indices where the new coordinates should be inserted
insert_idx = sorted(list(set(new_idx) - set(idx)))

# create new coordinate and value arrays
new_row = laplacians[1].row.tolist() + [coord[0] for coord in new_coords]
new_col = laplacians[1].col.tolist() + [coord[1] for coord in new_coords]
new_data = laplacians[1].data.tolist() + new_values

# insert the new coordinates and values into the existing matrix
laplacians[1] = sp.coo_matrix((new_data, (new_row, new_col)), shape=laplacians[1].shape)

# sort the matrix by row and column indices
laplacians[1] = sp.coo_matrix((laplacians[1].data[np.argsort(idx.tolist() + insert_idx)],
                               (laplacians[1].row[np.argsort(idx.tolist() + insert_idx)],
                                laplacians[1].col[np.argsort(idx.tolist() + insert_idx)])),
                              shape=laplacians[1].shape)


In [ ]:
print(type(laplacians[1]))

<class 'scipy.sparse._coo.coo_matrix'>


In [ ]:
laplacians[1].data[0]

6.0

In [ ]:
laplacians[1].shape

(1474, 1474)

In [ ]:
import random

random_number = random.random()
print(random_number)

0.20771364427792316


In [ ]:
# from torch._C import float32
for i in range(25466):
  laplacians[1].data[i] *= random_number


In [ ]:
for i in range(1474):
  laplacians[1].data[(laplacians[1].row==i) & (laplacians[1].col==i)]+=1


In [ ]:
print(laplacians[1])

  (0, 0)	2.2462819
  (0, 63)	-0.20771365
  (0, 64)	-0.20771365
  (0, 65)	-0.20771365
  (0, 66)	-0.20771365
  (0, 67)	-0.20771365
  (0, 68)	-0.20771365
  (0, 69)	-0.20771365
  (0, 70)	-0.20771365
  (0, 71)	-0.20771365
  (0, 72)	-0.20771365
  (0, 73)	-0.20771365
  (0, 74)	-0.20771365
  (0, 75)	-0.20771365
  (0, 76)	-0.20771365
  (0, 77)	-0.20771365
  (0, 78)	-0.20771365
  (0, 79)	-0.20771365
  (0, 80)	-0.20771365
  (0, 81)	-0.20771365
  (0, 82)	-0.20771365
  (0, 83)	-0.20771365
  (0, 84)	-0.20771365
  (0, 85)	-0.20771365
  (0, 86)	-0.20771365
  :	:
  (1471, 1471)	3.2848501
  (1471, 139)	0.20771365
  (1471, 470)	0.20771365
  (1471, 555)	0.20771365
  (1471, 804)	0.20771365
  (1471, 1201)	0.20771365
  (1471, 1357)	0.20771365
  (1471, 1373)	0.20771365
  (1471, 1408)	0.20771365
  (1471, 1473)	-0.20771365
  (1472, 1472)	2.4539955
  (1473, 29)	-0.20771365
  (1473, 154)	-0.20771365
  (1473, 470)	-0.20771365
  (1473, 555)	-0.20771365
  (1473, 771)	-0.20771365
  (1473, 804)	-0.20771365
  (1473, 10

In [ ]:
%cd /content/simplicial_neural_networks
np.save('150250_laplacians.npy', laplacians)


/content/simplicial_neural_networks


In [ ]:
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import inv
laplacians[1]=inv(laplacians[1])

/usr/local/lib/python3.10/dist-packages/scipy/sparse/linalg/_dsolve/linsolve.py:214: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',
/usr/local/lib/python3.10/dist-packages/scipy/sparse/linalg/_dsolve/linsolve.py:285: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  warn('spsolve is more efficient when sparse b '


In [ ]:
for i in range(1473):
  print(laplacians[1].getrow(i).getcol(i).toarray()[0][0])

0.3259099
0.25
0.25
0.25
0.25
0.18181819
0.18181819
0.18181819
0.18181819
0.20390324
0.18181819
0.18181819
0.18181819
0.2
0.2
0.2
0.2
0.26123047
0.2
0.2
0.15384616
0.16227514
0.15384616
0.15384616
0.15384616
0.20340891
0.15384616
0.16227515
0.15384616
0.17265742
0.18181819
0.21989362
0.18181819
0.18181819
0.18181819
0.18181819
0.18181819
0.20097613
0.31838745
0.25
0.25
0.25
0.25
0.18181819
0.18181819
0.18181819
0.20390315
0.18181819
0.18181819
0.18181819
0.2
0.2538233
0.22222213
0.2
0.2
0.2
0.2
0.18181819
0.18181819
0.20390323
0.18181819
0.18181819
0.18181819
0.373529
0.37352902
0.37352896
0.32590994
0.2406963
0.23500086
0.2592433
0.23500088
0.288873
0.43702102
0.24069634
0.23500088
0.23500088
0.28887293
0.37352896
0.65924335
0.5259099
0.37352893
0.33618373
0.300826
0.437021
0.288873
0.43702105
0.2592433
0.28887296
0.23500088
0.25924322
0.28887293
0.28887293
0.28887296
0.25924334
0.288873
0.2888729
0.6592433
0.23500088
0.25924334
0.2592433
0.32590994
0.2406963
0.32590997
0.2592433
0.26

In [ ]:
print(laplacians[1])

  (0, 0)	0.3259099
  (9, 0)	-0.00012907143
  (17, 0)	-8.5398024e-05
  (21, 0)	-0.000879516
  (25, 0)	0.0006843986
  (27, 0)	-0.0008795162
  (29, 0)	-0.0005503578
  (31, 0)	-0.00036336784
  (37, 0)	-0.0012107567
  (38, 0)	-1.1055643e-06
  (46, 0)	-0.00012907141
  (51, 0)	-8.5398024e-05
  (59, 0)	-0.00012907143
  (63, 0)	0.00742336
  (64, 0)	0.0074233664
  (65, 0)	0.0074233674
  (66, 0)	0.0074233618
  (67, 0)	0.008366763
  (68, 0)	0.0074233683
  (69, 0)	0.0074233674
  (70, 0)	0.0074233664
  (71, 0)	0.0074233594
  (72, 0)	0.007423363
  (73, 0)	0.008366767
  (74, 0)	0.007423368
  :	:
  (1364, 1473)	-0.0031911202
  (1365, 1473)	-0.0031911198
  (1366, 1473)	-0.00319112
  (1367, 1473)	-0.0031911207
  (1368, 1473)	-0.00319112
  (1369, 1473)	-0.0031911202
  (1370, 1473)	-0.0031911202
  (1371, 1473)	-0.0031911198
  (1372, 1473)	-0.0031911193
  (1373, 1473)	0.03946359
  (1397, 1473)	9.456306e-05
  (1408, 1473)	0.033879135
  (1411, 1473)	9.456306e-05
  (1421, 1473)	-0.0007806291
  (1431, 1473)	-0.

In [ ]:
def main():
    torch.manual_seed(1337)
    np.random.seed(1337)


    prefix = "./data/s2_3_collaboration_complex" ##input

    logdir = "./experiments/output" ##output
    starting_node="150250"
    percentage_missing_values="30"
    cuda = False

    topdim = 2
    %cd '/content/simplicial_neural_networks/data/s2_3_collaboration_complex'

    laplacians = np.load('{}_laplacians.npy'.format(starting_node),allow_pickle=True)
    boundaries = np.load('{}_boundaries.npy'.format(starting_node),allow_pickle=True)
    %cd /content/simplicial_neural_networks


    Ls =[coo2tensor(chebyshev.normalize(laplacians[i],half_interval=True)) for i in range(topdim+1)] #####scnn.chebyshev.normalize ?
    Ds=[coo2tensor(boundaries[i].transpose()) for i in range(topdim+1)]
    adDs=[coo2tensor(boundaries[i]) for i in range(topdim+1)]


    network = MySCNN(colors = 1)


    learning_rate = 0.001
    optimizer = torch.optim.Adam(network.parameters(), lr=learning_rate)
    criterion = nn.L1Loss(reduction="sum")
    #criterion = nn.MSELoss(reduction="sum")

    batch_size = 1

    num_params = 0
    print("Parameter counts:")
    for param in network.parameters():
        p = np.array(param.shape, dtype=int).prod()
        print(p)
        num_params += p
    print("Total number of parameters: %d" %(num_params))

    %cd /content/simplicial_neural_networks/data/s2_3_collaboration_complex
    masks_all_deg = np.load('{}_percentage_{}_known_values.npy'.format(starting_node,percentage_missing_values),allow_pickle=True) ## positive mask= indices that we keep ##1 mask #entries 0 degree
    masks=[list(masks_all_deg[i].values()) for i in range(len(masks_all_deg))]
    # losslogf = open("%s/loss.txt" %(logdir), "w")

    cochain_target_alldegs = []
    signal = np.load('{}_cochains.npy'.format(starting_node),allow_pickle=True)
    raw_data=[list(signal[i].values()) for i in range(len(signal))]
    for d in range(0, topdim+1):
        cochain_target = torch.zeros((batch_size, 1, len(raw_data[d])), dtype=torch.float, requires_grad = False)
        for i in range(0, batch_size):
            cochain_target[i, 0, :] = torch.tensor(raw_data[d], dtype=torch.float, requires_grad = False)

        cochain_target_alldegs.append(cochain_target)

    cochain_input_alldegs = []
    signal = np.load('{}_percentage_{}_input_damaged.npy'.format(starting_node,percentage_missing_values),allow_pickle=True)
    raw_data=[list(signal[i].values()) for i in range(len(signal))]
    for d in range(0, topdim+1):

        cochain_input = torch.zeros((batch_size, 1, len(raw_data[d])), dtype=torch.float, requires_grad = False)

        for i in range(0, batch_size):
            cochain_input[i, 0, :] = torch.tensor(raw_data[d], dtype=torch.float, requires_grad = False)

        cochain_input_alldegs.append(cochain_input)

    #cochain_target_alldegs[0] = torch.zeros_like(cochain_target_alldegs[0])
    #cochain_target_alldegs[2] = torch.zeros_like(cochain_target_alldegs[2])

    #cochain_input_alldegs[0] = torch.zeros_like(cochain_input_alldegs[0])
    #cochain_input_alldegs[2] = torch.zeros_like(cochain_input_alldegs[2])

    print([float(len(masks[d]))/float(len(cochain_target_alldegs[d][0,0,:])) for d in range(0,2+1)])

    for i in range(0, 1000):
        xs = [cochain_input.clone() for cochain_input in cochain_input_alldegs]

        optimizer.zero_grad()
        ys = network(Ls, Ds, adDs, xs)

        loss = torch.FloatTensor([0.0])
        for b in range(0, batch_size):
            for d in range(0, topdim+1):
                loss += criterion(ys[d][b, 0, masks[d]], cochain_target_alldegs[d][b, 0, masks[d]])

        detached_ys = [ys[d].detach() for d in range(0, topdim+1)]

        # if np.mod(i, 10) == 0:
        #     for d in range(0,topdim+1):
        #         np.savetxt("%s/output_%d_%d.txt" %(logdir, i, d), detached_ys[d][0,0,:])

        for d in range(0, topdim+1):
            predictionlogf = open("%s/prediction_%d_%d.txt" %(logdir, i, d), "w")
            actuallogf = open("%s/actual_%d_%d.txt" %(logdir, i, d), "w")

            for b in range(0, batch_size):
                for y in detached_ys[d][b, 0, masks[d]]:
                    predictionlogf.write("%f " %(y))
                predictionlogf.write("\n")
                for x in cochain_target_alldegs[d][b, 0, masks[d]]:
                    actuallogf.write("%f " %(x))
                actuallogf.write("\n")
            predictionlogf.close()
            actuallogf.close()


        losslogf.write("%d %f\n" %(i, loss.item()))
        losslogf.flush()

        loss.backward()
        optimizer.step()

    # losslogf.close()

    name_networks=['C0_1,C0_2','C0_3','C1_1,C1_2','C1_3', 'C2_1,C2_2','C2_3']



if __name__ == "__main__":
    main()

/content/simplicial_neural_networks/data/s2_3_collaboration_complex
/content/simplicial_neural_networks
Parameter counts:
150
30
4500
30
150
1
150
30
4500
30
150
1
150
30
4500
30
150
1
Total number of parameters: 14583
/content/simplicial_neural_networks/data/s2_3_collaboration_complex
[0.6988636363636364, 0.6994572591587517, 0.6998477929984779]


NameError: ignored

In [ ]:
name_networks=['C0_1,C0_2','C0_3','C1_1,C1_2','C1_3', 'C2_1,C2_2','C2_3']



if __name__ == "__main__":
    main()

/content/simplicial_neural_networks/data/s2_3_collaboration_complex
/content/simplicial_neural_networks
Parameter counts:
150
30
4500
30
150
1
150
30
4500
30
150
1
150
30
4500
30
150
1
Total number of parameters: 14583
/content/simplicial_neural_networks/data/s2_3_collaboration_complex
[0.6988636363636364, 0.6994572591587517, 0.6998477929984779]


KeyboardInterrupt: ignored